In [1]:
import os
import pickle
import gc

import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.regularizers import L1L2

from sklearn.model_selection import train_test_split

In [2]:
MODEL = "7-auto-regressive-lstm-covariates-future-only"
COVARIATE_COLUMNS = ['covs_week_sin','covs_week_cos','covs_day_sin','covs_day_cos','covs_hour_sin','covs_hour_cos','covs_rbf_month_1','covs_rbf_month_2','covs_rbf_month_3','covs_rbf_month_4','covs_rbf_month_5','covs_rbf_month_6','covs_rbf_month_7','covs_rbf_month_8','covs_rbf_month_9','covs_rbf_month_10','covs_rbf_month_11','covs_rbf_month_12']

config = {
    "N_PREV": 120,
    "N_FORWARD": 48,
    "STEP_SIZE": 1,
    
    "LATENT_SIZE": 16,
    
    "LR": 0.001,
    "BATCH_SIZE": 256,
    "EPOCHS": 2,
    
    "L1": 0.0000001, 
    "L2": 0.0000001,
    
    "TARGET_COL": 'normalized_level',
}

In [3]:
import wandb
from wandb.keras import WandbCallback
from secrets import WANDB
wandb.login(key=WANDB)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\KiernanMcGuigan/.netrc


True

## **Load Data Function**

In [4]:
def get_samples(df, set_number):
    subset = df.loc[df.set==set_number, :]
    shape = subset.shape
    if(shape[0] > 1_000):
        subset = subset.iloc[-1_000:, :]
    data = subset[config['TARGET_COL']].values
    covariates = subset[COVARIATE_COLUMNS].values
    print(f'Data Shape: {shape}, Reduced Shape {data.shape}')
    
    X, y, future_covs, dates = [], [], [], []
    for start_idx in range(0, data.shape[0]-config["N_PREV"]-config["N_FORWARD"], config["STEP_SIZE"]):
        mid_idx = start_idx + config["N_PREV"]
        end_idx = mid_idx + config["N_FORWARD"]
        X.append(data[start_idx:mid_idx])
        y.append(data[mid_idx:end_idx])
        future_covs.append(covariates[mid_idx:end_idx])
        dates.append(subset.iloc[mid_idx:end_idx].index.values)
    return np.array(X), np.array(y), np.array(future_covs), np.array(dates)

def to_dataset(X1, X2, y):
    dataset = tf.data.Dataset.from_tensor_slices(({'series':X1, 'future_dates':X2}, y))
    dataset = dataset.cache().shuffle(X1.shape[0]+1).batch(config["BATCH_SIZE"]).prefetch(tf.data.experimental.AUTOTUNE)
    return dataset

def get_train_and_val_datasets(X1, X2, y):
    X1_train, X1_val, X2_train, X2_val, y_train, y_val = train_test_split(X1, X2, y, test_size=0.15, shuffle=True)
    train_ds = to_dataset(X1_train, X2_train, y_train)
    val_ds = to_dataset(X1_val, X2_val, y_val)
    return train_ds, val_ds

def df_to_samples(df):
    X, y, future, _ = get_samples(df, 0)
    train_ds, val_ds = get_train_and_val_datasets(X, future, y)
    X_test, y_test, future_test, dates = get_samples(df, 1)
    return train_ds, val_ds, X_test, y_test, future_test, dates
    
def get_data(fold):
    df = pd.read_csv(f'data/fold{fold}_data_v3.csv')
    with open(f'./data/fold{fold}_normalizer_v3.pkl', mode='rb') as f:
        normalizer = pickle.load(f)
    return df_to_samples(df), normalizer

## **Model Functions**

In [7]:
def get_model():
    inputs = tf.keras.layers.Input(shape=(config["N_PREV"], 1), name='series')
    future_covs = tf.keras.layers.Input(shape=(config["N_FORWARD"], len(COVARIATE_COLUMNS)), name='future_dates')
    
    lstm1 = tf.keras.layers.LSTM(config["LATENT_SIZE"], 
                                 return_sequences=True, 
                                 return_state=True, 
                                 recurrent_initializer='glorot_uniform', 
                                 activity_regularizer=L1L2(l1=config['L1'],l2=config['L2']))
    lstm2 = tf.keras.layers.LSTM(config["LATENT_SIZE"], 
                                 return_sequences=True, 
                                 return_state=True, 
                                 recurrent_initializer='glorot_uniform', 
                                 activity_regularizer=L1L2(l1=config['L1'],l2=config['L2']))
    lstm3 = tf.keras.layers.LSTM(config["LATENT_SIZE"], 
                                 return_sequences=False, 
                                 return_state=True, 
                                 recurrent_initializer='glorot_uniform', 
                                 activity_regularizer=L1L2(l1=config['L1'],l2=config['L2']))
    dense1 = tf.keras.layers.Dense(16)
    dense2 = tf.keras.layers.Dense(8)
    dense3 = tf.keras.layers.Dense(1)
    
    predictions = []
    x, hidden_state1, cell_state1 = lstm1(inputs)
    x, hidden_state2, cell_state2 = lstm2(x)
    x, hidden_state3, cell_state3 = lstm3(x)
    x = K.concatenate([x, future_covs[:, 0, :]])
    x = dense1(x)
    x = dense2(x)
    x = dense3(x)
    
    predictions.append(x)
    for idx in range(1, config['N_FORWARD']):
        x = tf.expand_dims(x, axis=1)
        x, hidden_state1, cell_state1 = lstm1(x, initial_state=[hidden_state1, cell_state1])
        x, hidden_state2, cell_state2 = lstm2(x, initial_state=[hidden_state2, cell_state2])
        x, hidden_state3, cell_state3 = lstm3(x, initial_state=[hidden_state3, cell_state3])
        x = K.concatenate([x, future_covs[:, idx, :]])
        x = dense1(x)
        x = dense2(x)
        x = dense3(x)
        
        predictions.append(x)
        
    outputs = tf.keras.layers.Lambda(lambda x: K.concatenate(x, axis=1))(predictions)
    model = tf.keras.models.Model(inputs=[inputs,future_covs], outputs=outputs)
    model.compile(loss="mse", metrics=["mae"], optimizer=tf.keras.optimizers.Adam(learning_rate=config["LR"]))
    return model

# def get_model():
#     model = MyModel()
#     model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(config["LR"]), metrics=['mae'])
#     model.predict(tf.ones(shape=(config['BATCH_SIZE'],config['N_PREV'],1)))
#     return model
    
test_model = get_model()
print(test_model.summary())
del test_model
gc.collect()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 series (InputLayer)            [(None, 120, 1)]     0           []                               
                                                                                                  
 lstm_6 (LSTM)                  multiple             1152        ['series[0][0]',                 
                                                                  'tf.expand_dims_47[0][0]',      
                                                                  'lstm_6[0][1]',                 
                                                                  'lstm_6[0][2]',                 
                                                                  'tf.expand_dims_48[0][0]',      
                                                                  'lstm_6[1][1]',           

                                                                  'lstm_6[25][2]',                
                                                                  'tf.expand_dims_73[0][0]',      
                                                                  'lstm_6[26][1]',                
                                                                  'lstm_6[26][2]',                
                                                                  'tf.expand_dims_74[0][0]',      
                                                                  'lstm_6[27][1]',                
                                                                  'lstm_6[27][2]',                
                                                                  'tf.expand_dims_75[0][0]',      
                                                                  'lstm_6[28][1]',                
                                                                  'lstm_6[28][2]',                
          

                                                                  'lstm_7[5][2]',                 
                                                                  'lstm_6[7][0]',                 
                                                                  'lstm_7[6][1]',                 
                                                                  'lstm_7[6][2]',                 
                                                                  'lstm_6[8][0]',                 
                                                                  'lstm_7[7][1]',                 
                                                                  'lstm_7[7][2]',                 
                                                                  'lstm_6[9][0]',                 
                                                                  'lstm_7[8][1]',                 
                                                                  'lstm_7[8][2]',                 
          

                                                                  'lstm_7[33][1]',                
                                                                  'lstm_7[33][2]',                
                                                                  'lstm_6[35][0]',                
                                                                  'lstm_7[34][1]',                
                                                                  'lstm_7[34][2]',                
                                                                  'lstm_6[36][0]',                
                                                                  'lstm_7[35][1]',                
                                                                  'lstm_7[35][2]',                
                                                                  'lstm_6[37][0]',                
                                                                  'lstm_7[36][1]',                
          

                                                                  'lstm_8[12][2]',                
                                                                  'lstm_7[14][0]',                
                                                                  'lstm_8[13][1]',                
                                                                  'lstm_8[13][2]',                
                                                                  'lstm_7[15][0]',                
                                                                  'lstm_8[14][1]',                
                                                                  'lstm_8[14][2]',                
                                                                  'lstm_7[16][0]',                
                                                                  'lstm_8[15][1]',                
                                                                  'lstm_8[15][2]',                
          

...

                                                                                                  
 tf.concat_66 (TFOpLambda)      (None, 34)           0           ['lstm_8[18][0]',                
                                                                  'tf.__operators__.getitem_66[0][
                                                                 0]']                             
                                                                                                  
 tf.expand_dims_65 (TFOpLambda)  (None, 1, 1)        0           ['dense_8[18][0]']               
                                                                                                  
 tf.__operators__.getitem_67 (S  (None, 18)          0           ['future_dates[0][0]']           
 licingOpLambda)                                                                                  
                                                                                                  
 tf.concat

                                                                  'tf.__operators__.getitem_75[0][
                                                                 0]']                             
                                                                                                  
 tf.expand_dims_74 (TFOpLambda)  (None, 1, 1)        0           ['dense_8[27][0]']               
                                                                                                  
 tf.__operators__.getitem_76 (S  (None, 18)          0           ['future_dates[0][0]']           
 licingOpLambda)                                                                                  
                                                                                                  
 tf.concat_76 (TFOpLambda)      (None, 34)           0           ['lstm_8[28][0]',                
                                                                  'tf.__operators__.getitem_76[0][
          

                                                                                                  
 tf.expand_dims_83 (TFOpLambda)  (None, 1, 1)        0           ['dense_8[36][0]']               
                                                                                                  
 tf.__operators__.getitem_85 (S  (None, 18)          0           ['future_dates[0][0]']           
 licingOpLambda)                                                                                  
                                                                                                  
 tf.concat_85 (TFOpLambda)      (None, 34)           0           ['lstm_8[37][0]',                
                                                                  'tf.__operators__.getitem_85[0][
                                                                 0]']                             
                                                                                                  
 tf.expand

                                                                                                  
 tf.__operators__.getitem_94 (S  (None, 18)          0           ['future_dates[0][0]']           
 licingOpLambda)                                                                                  
                                                                                                  
 tf.concat_94 (TFOpLambda)      (None, 34)           0           ['lstm_8[46][0]',                
                                                                  'tf.__operators__.getitem_94[0][
                                                                 0]']                             
                                                                                                  
 tf.expand_dims_93 (TFOpLambda)  (None, 1, 1)        0           ['dense_8[46][0]']               
                                                                                                  
 tf.__oper

514951

## **Train Loop**

In [11]:
def train_model(model, train_ds, val_ds):
    reducer = tf.keras.callbacks.ReduceLROnPlateau(monior='val_loss', factor=0.1, patience=2, mode='min', cooldown=1)
    stopper = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4, mode='min', restore_best_weights=True)
    model.fit(train_ds,
              epochs=config["EPOCHS"], 
              callbacks=[reducer, stopper, WandbCallback()],
              validation_data=val_ds)
    return model

def test_model(model, X_test, y_test, future_test, normalizer):
    predictions = model.predict({'series':X_test, 'future_dates':future_test})
    inversed_predictions, inversed_actuals = [], []
    for pred, actual in zip(predictions, y_test):
        inversed_predictions.append(normalizer.inverse_transform(np.array(pred).reshape(-1,1)).reshape((-1)))
        inversed_actuals.append(normalizer.inverse_transform(np.array(actual).reshape(-1,1)).reshape((-1)))
    inversed_predictions = np.array(inversed_predictions)
    inversed_actuals = np.array(inversed_actuals)
    forward_error = np.mean(np.abs(inversed_actuals - inversed_predictions), axis=-1)
    error = np.mean(forward_error)
    return error, forward_error, inversed_predictions, inversed_actuals

def record_forward_error(forward_errors, error, run):
    run.log({'test/error': error})
    run.log({'test/forward_error': wandb.plot.line_series(
        xs=[i+1 for i in range(forward_errors.shape[0])],
        ys=[forward_errors],
        keys=['Forward Horizon Errors'],
        title=f'Forward Horizon Errors (MAE: {error})',
    )})
        
def graph_predictions(predictions, actuals, dates, run):
    for look_ahead in [1, 12, 24, 36, 48]:
        preds, true, time = predictions[:, look_ahead-1], actuals[:, look_ahead-1], dates[:, look_ahead-1]
        error = np.mean(np.abs(true - preds))
        run.log({f'test/forecast-{look_ahead}-hours-ahead': wandb.plot.line_series(
            xs=time,
            ys=[true, preds],
            keys=['Actuals', 'Predictions'],
            title=f'{look_ahead} Hour(s) Ahead Forecast (MAE: {error})',
            xname='week'
        )})

In [12]:
def train():
    for fold in range(1, 4):
        run = wandb.init(project="time-series-methods", entity="kmcguigan", group=f"{MODEL}-model", config=config, job_type="train")
        run.name = f'{MODEL}-fold-{fold}'
        (train_ds, val_ds, X_test, y_test, future_test, dates), normalizer = get_data(fold)
        model = get_model()
        model = train_model(model, train_ds, val_ds)
        error, forward_errors, predictions, actuals = test_model(model, X_test, y_test, future_test, normalizer)
        run.log({'test-error': error})
        record_forward_error(forward_errors, error, run)
        graph_predictions(predictions, actuals, dates, run)
        run.finish()
        del model
        del train_ds
        del val_ds
        gc.collect()
    return

In [ ]:
train()

Data Shape: (159096, 23), Reduced Shape (1000,)
Data Shape: (43390, 23), Reduced Shape (1000,)


wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/2
3/3 [==============================] - 288s 32s/step - loss: 1.3729 - mae: 0.9682 - val_loss: 1.2285 - val_mae: 0.9059 - lr: 0.0010 - _timestamp: 1654479680.0000 - _runtime: 332.0000
Epoch 2/2
26/26 [==============================] - 56s 103ms/step


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


epoch,▁█
loss,█▁
lr,▁▁
mae,█▁
test-error,▁
test/error,▁
val_loss,█▁
val_mae,█▁
best_epoch,1
best_val_loss,1.1296
epoch,1


Data Shape: (202486, 23), Reduced Shape (1000,)
Data Shape: (43390, 23), Reduced Shape (1000,)
Epoch 1/2
3/3 [==============================] - 290s 29s/step - loss: 2.4602 - mae: 1.3094 - val_loss: 2.3039 - val_mae: 1.2609 - lr: 0.0010 - _timestamp: 1654480115.0000 - _runtime: 333.0000
Epoch 2/2
26/26 [==============================] - 55s 92ms/step


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


wandb: ERROR Control-C detected -- Run data was not synced
Exception ignored in: <function ScopedTFFunction.__del__ at 0x000001E52CCA1F78>
Traceback (most recent call last):
  File "C:\Users\KiernanMcGuigan\.conda\envs\ml-ts-env\lib\site-packages\tensorflow\python\framework\c_api_util.py", line 122, in __del__
    self.deleter(func)
KeyboardInterrupt
Exception ignored in: <function _EagerDefinedFunctionDeleter.__del__ at 0x000001E52EDE4708>
Traceback (most recent call last):
  File "C:\Users\KiernanMcGuigan\.conda\envs\ml-ts-env\lib\site-packages\tensorflow\python\eager\function.py", line 304, in __del__
    context.remove_function(self.name)
  File "C:\Users\KiernanMcGuigan\.conda\envs\ml-ts-env\lib\site-packages\tensorflow\python\eager\context.py", line 2719, in remove_function
    context().remove_function(name)
KeyboardInterrupt: 
